<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Librerías" data-toc-modified-id="Librerías-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Librerías</a></span></li><li><span><a href="#Base-de-Datos---Airlines" data-toc-modified-id="Base-de-Datos---Airlines-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Base de Datos - Airlines</a></span></li><li><span><a href="#Conclusiones" data-toc-modified-id="Conclusiones-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Conclusiones</a></span></li></ul></div>

# Análisis de Sentimiento - Clasificación de Texto

En este notebook se pretende realizar un __análisis de sentimiento (clasificación de texto)__ para dos conjuntos de datos distintos. 

Este notebook hace parte (y es el comienzo) de la serie de notebooks y scripts del curso de __Tópicos de Estadística Avanzada (DS y ML)__ tomado en la Universiad Nacional de Colombia.

## Librerías 

En esta seccion se importan las librerías y modulos que se van a usar a lo largo del notebook.

In [4]:
# Se importan modelos y funciones param mostrar imagenes
from IPython.display import Image
from IPython.core.display import HTML 

# DS y ML
import pandas as pd
import numpy as np
from wordcloud import WordCloud,STOPWORDS

## Base de Datos - Airlines

En este primer capítulo se va a realizar el análisis de sentimiento para el conjunto de datos `../Data/Tweets/AirlinesTweets.csv` el cual contiene tweets acerca la opiniones de distintas aereolíneas, para más información ir [aquí](https://www.kaggle.com/crowdflower/twitter-airline-sentiment?select=Tweets.csv).

In [5]:
# Lea el .csv de tweets
pd_tweets = pd.read_csv('../Data/Tweets/AirlinesTweets.csv', sep=',')
pd_tweets.head(n=2)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)


Lo normal es hacer un fuerte __análisis descriptivo y exploratorio__ de los datos que se tienen para cada una de las aereolíneas y los cada uno de los tipos de sentimiento (positivo, negativo y neutro).

Por ahora vamos a ver la nube de palabras para cada tipo de sentimiento para ver si de manera general se tienen algunas diferencias. 

In [ ]:
sentiments = pd_tweets['']

words = ' '.join(df['text'])
# cleaned_word = " ".join([word for word in words.split()
#                             if 'http' not in word
#                                 and not word.startswith('@')
#                                 and word != 'RT'
#                             ])

## Conclusiones

`### TO DO`

# Final del Documento